In [2]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt
import json
import pickle

file_name = "BiLSTM_module_ver1.0"

# GPU 있는 경우 활성화
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

with open(file_name+'_variable.json') as f:
    var = json.load(f)

max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

model = tf.keras.models.load_model(file_name+'.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = ['\n','.','?',',','']
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    print(data)
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    if (score > 0.5):
        print(' {:.2f}% 확률로 스팸입니다.'.format(score * 100))
    else:
        print(' {:.2f}% 확률로 스팸이 아닙니다.'.format((1 - score) * 100))
    
    print('-------------------------------------------------------------------------')
        

# 음성파일 STT 변환
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile('ham1.wav') # 100MB 용량 제한
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')

temp_X = []
x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한
for s in x:
    if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
        temp_X.append(s)
x = temp_X
for txt in x:
    print(txt)
    data = preTreatment(txt)
    sentiment_predict(data)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 60)          289260    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               128800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 418,261
Trainable params: 418,261
Non-trainable params: 0
_________________________________________________________________


FileNotFoundError: [Errno 2] No such file or directory: 'ham1.wav'